# モデルの構築

## ニューラルネットワークのモデルの作成
ニューラルネットワークは、**レイヤ(モジュール)**と呼ばれるデータ操作の入れ子で構成される。  
特に、pytorchでは、`torch.nn`で独自のニューラルネットワークを構築するためのクラスや関数が揃っている。  
また、pytorchの全てのモジュールは、`torch.nn.Module`を継承している。  
このような入れ子構造により、複雑な構造を簡単に構築・管理することができる。

以下、FashionMNISTを分類するネットワークを構築する。

In [ ]:
%matplotlib inline

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## CPUかGPUか
GPUが使用できる環境であレバ、GPUを用いる方が良い。(高速ゆえ)  
GPUが使用可能かどうかは、`torch.cuda`の結果から判断が可能。  
GPUが使用可能でない場合、CPUを用いて学習を行う。

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

## モデルのクラスの定義
`nn.Module`を継承させ、独自のネットワークモデルを定義し、  
その後、`__init__`により、初期化する。  
`nn.Module`を継承した全てのモジュールは、順伝播のための`forward`関数を持たなければならない。

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

## クラスの定義後
作成したモデルのインスタンスを作成し、  
**変数`device`上へと移動**させる

## モデルに入力を渡す
入力データをモデルに渡すと**自動的に**`forward`関数で処理されるとともに、  
いくつか`background.operations`が実行される。  
そのため、**`model.forward()`**と書いて直接えーたを処理しないように注意する。  

以下、今回のモデルでは  
各クラスの生の予測値を含む10次元のテンソルが返されるため、  
`nn.SOftmax`モジュールにこの結果を与えることで、各クラスに属する確率の予測値を出力する

In [ ]:
model = NeuralNetwork().to(device)
print(model)

In [ ]:
# 仮の入力データの作成
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_logits = nn.Softmax(dim=1)(logits) # 確率を計算
y_pred = pred_logits.argmax(1) # 最も確率の高いものを予測結果へ
print(f"Predicted class : {y_pred}")


## 各モデルレイヤの確認
一つ一つのレイヤを順を追って解説。僕は飛ばします。
[元記事](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_4_buildmodel_tutorial_js.ipynb)参照

## モデルパラメータ
ニューラルネットワークを構成する多くのモジュールはそれぞれに重みが存在することが多く、  
重みやバイアスなど。学習時には、これらを最適化する。

`nn.Module`を継承したモデルオブジェクトで定義された全てのフィールドは、動的に追跡することができる。  
`parameters()`や、`named_parameters()`メソッドを用いて、  
モデルの各レイヤの全てのパラメータにアクセスすることができる。  

以下、for文を用いて、各パラメータを追跡する

In [ ]:
print(f"Model stracture : {model} \n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

## より詳細な情報
より詳細なページは[torch.nn API](https://pytorch.org/docs/stable/nn.html)を参照